Quick script to convert heights from Ellipsoidal to Orthometric. This was done
for a specific project as part of a course on Lidar applications.
The orthometric heights match the results from the GPS-H converter:
https://webapp.csrs-scrs.nrcan-rncan.gc.ca/geod/tools-outils/gpsh.php

In [1]:
import pyproj
from pyproj import CRS, Transformer
import pandas as pd

In [2]:
data = pd.read_csv('./data/SC_CTRL.csv')

In [3]:
data

,ID,Northing,Easting,Ellip_height
0,T17,4929648.096,397776.198,-21.227
1,T16,4929645.029,397773.015,-21.293
2,T15,4929637.751,397758.423,-20.356
3,T14,4929652.870,397742.188,-19.639
4,T13,4929664.401,397737.112,-19.869
5,T12,4929673.802,397735.326,-19.939
6,T11,4929680.582,397734.376,-20.076
7,T10,4929696.489,397731.401,-19.322
8,T9,4929691.047,397746.733,-21.248
9,T8,4929626.535,397768.207,-20.263


In [4]:
easting = data['Easting']
northing = data['Northing']
ellip_height = data['Ellip_height']


In [5]:
# 1- transform the horizontal coord from projected to ellip:

# NAD83(CSRS)v6 / UTM zone 20N
# https://epsg.org/crs_22620/NAD83-CSRS-v6-UTM-zone-20N.html
from_crs = CRS.from_epsg(22620)  

# NAD83(CSRS)
# https://epsg.org/crs_4955/NAD83-CSRS.html
to_crs = CRS.from_epsg(4955)

transformer = pyproj.Transformer.from_crs(from_crs, to_crs)

# transform from UTM / ellipsoidal to lat/lon/h (height same)
lat, lon, eh = transformer.transform(easting, northing, ellip_height)

# check results:
print(f"Easting -> Lat: {easting[4]} -> {lat[4]}")
print(f"Northing -> Long: {northing[4]} -> {lon[4]}")
print(f"Ellip height -> Ellip height (not transformed): {ellip_height[4]} -> {eh[4]}")


Easting -> Lat: 397737.112 -> 44.513055446081275
Northing -> Long: 4929664.401 -> -64.28664717574767
Ellip height -> Ellip height (not transformed): -19.869 -> -19.869


In [6]:
# 2- now transform the horizontal coord back to projected (UTM)
# and the ellipsidal height to the ortho heights

# NAD83(CSRS)
# https://epsg.org/crs_4955/NAD83-CSRS.html
from_crs = CRS.from_epsg(4955)

# NAD83(CSRS) / UTM zone 20N + CGVD2013 height
# https://epsg.org/crs_6663/NAD83-CSRS-UTM-zone-20N-CGVD2013-height.html
to_crs = CRS.from_epsg(6663)

transformer = pyproj.Transformer.from_crs(from_crs, to_crs)

# transform from UTM / ellipsoidal to lat/lon/h (height same)
e, n, oh = transformer.transform(lat, lon, eh)

# check results:
print(f"Easting -> Lat: {e[7]} -> {lat[7]}")
print(f"Northing -> Lon: {n[7]} -> {lon[7]}")
print(f"Ellip height -> Ortho height: {eh[7]} -> {oh[7]}")


Easting -> Lat: 397731.40099999966 -> 44.51334344227972
Northing -> Lon: 4929696.489 -> -64.28672536801889
Ellip height -> Ortho height: -19.322 -> 2.092527047840182


In [7]:
# write the new heights to file:
new_data = data.copy()
new_data['ortho_heights'] = oh


In [8]:
# drop ellip height
new_data = new_data.drop(columns=['Ellip_height'])

In [9]:
# reorder columns to match format expected by Faro Scene
new_data = new_data[['ID', 'Easting', 'Northing', 'ortho_heights']]

new_data

,ID,Easting,Northing,ortho_heights
0,T17,397776.198,4929648.096,0.187170
1,T16,397773.015,4929645.029,0.121179
2,T15,397758.423,4929637.751,1.058238
3,T14,397742.188,4929652.870,1.775361
4,T13,397737.112,4929664.401,1.545416
5,T12,397735.326,4929673.802,1.475449
6,T11,397734.376,4929680.582,1.338471
7,T10,397731.401,4929696.489,2.092527
8,T9,397746.733,4929691.047,0.166433
9,T8,397768.207,4929626.535,1.151158


In [10]:
# save to file. Drop the last 3 lines (not part of the data)
new_data[:-3].to_csv('./data/formated_sc_ctrl.csv', 
                     index=False,
                     header=False)